# Rotator Duty Cycle Test - Regular Observing Campaign

Use this notebook to run the [LVV-T2572 - Rotator Duty Cycle] test case.

[LVV-T2572 - Rotator Duty Cycle]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2572

Most of the time during operations the scheduler will be imaging a pack of the sky trying to keep the rotator close to a certain position.  
Basically, we can assume it is going to “slew” the rotator to a certain position and, from there on, track and unwind the rotator constantly for hours on end.  
From the rotator perspective, this can be seen as something like:

 - Test sequence: The rotator controller must be set into tracking / slewing operation.  
   For this, the commands used in `dqueue_test_trackingSlewing` can be used.  
   Once in tracking / slewing operation mode, the following setpoints must be sent:  
   - Start with rotator position in zero.  
   - Repeat 240 times (around 2 hours): Track target with starting Rotator position around -0.3 deg and tracking velocity +0.01deg/s for 30s.  


 - Requirements and flags validated during this test.  
   This test is used to validate the following requirements (for more information, see section 3.1):  
   - ID 3.4.10 – Rotator duty cycle  


 - Results: The setpoint time, position and speed must be compared with the generated trajectory output.   
   It must also be compared with the feedback position and speed.   
   Both generated trajectory and feedback data should follow the desired setpoint, meeting the dynamic constraints of the rotator controller.  

## Who/When/Where?

The code below prints the user, time, and if you are at the summit or not.  
It is used mostly for tracking purposes.

In [ ]:
from lsst.ts import utils

# Extract your name from the Jupyter Hub
__executed_by__ = os.environ["JUPYTERHUB_USER"]  

# Extract execution date
__executed_on__ = utils.astropy_time_from_tai_unix(utils.current_tai())
__executed_on__.format = "isot"

# This is used later to define where Butler stores the images
summit = os.environ["LSST_DDS_PARTITION_PREFIX"] == "summit"

print(f"\nExecuted by {__executed_by__} on {__executed_on__}."
      f"\n  At the summit? {summit}")

## Set up

Before you run this notebook, make sure that you setup the Rotator and the CCW.  
The easiest way of doing this would be to go through [LVV-T????] test case and stop right after setting up the Rotator.  

The following cells contain the libraries that should be imported.  
Once imported, we can setup the log, start the domains, and add the components.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ.get("LSST_DDS_DOMAIN_ID", "Expected, not set."))

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
import os
import sys
import asyncio
import logging

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

import lsst.daf.butler as dafButler

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
index = os.getegid() + np.random.randint(-5, 5)

start_time = datetime.now()
script = salobj.Controller("Script", index=index)

## Tracking Speed

The tracking speed depends on the position of the target in the sky.  
Use the map below for a roght estimative of the required position to meet a given tracking speed.

![Rotator Tracking Velocity]("img/rotator-tracking-velocity.png")

## Test: Extreme Conditions

The idea of this test is to check different rapidly cycling the Rotator between “fast-tracking”, “slow-tracking”, tracking at different directions with big slews and small slews in between.  
These are conditions that will seldom occur during a regular observing night but are conditions we expect the rotator to be able to handle seamlessly.  

In [ ]:
now = datetime.now()
test_message = "LVV-T2572-A - Extreme Conditions"
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")

---
Start with rotator position in 0.

In [ ]:
await mtcs.rem.mtrotator.cmd_move.set_start(position=0.0)

Track target with starting Rotator position -80 deg with velocity +0.01 deg/s for 30s.
Final position: ~ -79.5 deg (assumes ~0.2 deg of “tracking” during the time it takes to slew from 0 to -80).

In [ ]:
target = await mtcs.find_target(el=60, az=180, mag_limit=8)  # Position w/ +0.01 deg/s
print(f"Found target near {el} deg elevation and {az} deg azimuth:\n"
      f"  {target}")

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=-80)
await asyncio.sleep(30)
await mtcs.stop_tracking()

Track target with starting Rotator position -80 deg with velocity +0.02 deg/s for 30s
Final position: ~ -79.4 deg (assuming slewing from -79.4 deg to -80 deg takes negligible amount of time).

In [ ]:
# Maximum elevation = 86.5 deg
target = await mtcs.find_target(el=85, az=180, mag_limit=8)  # Position w/ +0.02 deg/s
print(f"Found target near {el} deg elevation and {az} deg azimuth:\n"
      f"  {target}")

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=-80)
await asyncio.sleep(30)
await mtcs.stop_tracking()

Track target with starting Rotator position -80 deg with velocity -0.02 deg/s for 30s
Final position: ~-80.6 deg.

In [ ]:
target = await mtcs.find_target(el=85, az=0, mag_limit=8)  # Position w/ -0.02 deg/s
print(f"Found target near {el} deg elevation and {az} deg azimuth:\n"
      f"  {target}")

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=-80) 
await asyncio.sleep(30)
await mtcs.stop_tracking()

Track target with starting Rotator position -80 deg with velocity -0.01 deg/s for 30s
Final position: ~-80.3 deg.

In [ ]:
target = await mtcs.find_target(el=60, az=180, mag_limit=8)  # Position w/ -0.01 deg/s
print(f"Found target near {el} deg elevation and {az} deg azimuth:\n"
      f"  {target}")

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=-80)
await asyncio.sleep(30)
await mtcs.stop_tracking()

Track target with starting Rotator position +80 deg with velocity +0.01 deg/s for 30s
Final position: ~+80.7 deg (assumes ~0.4 deg of “tracking” during the time it takes to slew from ~-80.3 to +80).

In [ ]:
target = await mtcs.find_target(el=60, az=180, mag_limit=8) # Position w/ +0.01 deg/s
print(f"Found target near {el} deg elevation and {az} deg azimuth:\n"
      f"  {target}")

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=+80)
await asyncio.sleep(30)
await mtcs.stop_tracking()

Track target with starting Rotator position +80 deg with velocity +0.02 deg/s for 30s
Final position: ~+80.6 deg.

In [ ]:
target = await mtcs.find_target(el=85, az=180, mag_limit=8) # Position w/ +0.01 deg/s
print(f"Found target near {el} deg elevation and {az} deg azimuth:\n"
      f"  {target}")

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=+80)
await asyncio.sleep(30)
await mtcs.stop_tracking()

Track target with starting Rotator position +80 deg with velocity -0.02 deg/s for 30s
Final position: ~+79.4 deg.

In [ ]:
target = await mtcs.find_target(el=85, az=0, mag_limit=8) # Position w/ -0.02 deg/s
print(f"Found target near {el} deg elevation and {az} deg azimuth:\n"
      f"  {target}")

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=+80)
await asyncio.sleep(30)
await mtcs.stop_tracking()

Track target with starting Rotator position +80 deg with velocity -0.01 deg/s for 30s
Final position: ~+ 79.7 deg.

In [ ]:
target = await mtcs.find_target(el=60, az=0, mag_limit=8) # Position w/ -0.02 deg/s
print(f"Found target near {el} deg elevation and {az} deg azimuth:\n"
      f"  {target}")

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=+80)
await asyncio.sleep(30)
await mtcs.stop_tracking()

End of this part.

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- Starting Time: {now} UTC")

## Emulate filter change, tracking target with zero velocity between two targets.

There are some conditions we expect to be able to track a target through the sky while not rotating.  
These will happen while doing filter changes and also during some mount tracking tests we may perform during commissioning.  
At the very least, the rotator must be able to track a non-moving target for 2 minutes without interruption, which is the approximate time it takes to perform a filter change.

In [ ]:
now = datetime.now()
test_message = "LVV-T2572-B - Emulate Filter Change"
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")

---
Start with rotator position in zero.

In [ ]:
await mtcs.rem.mtrotator.cmd_move.set_start(position=0.0)

Track target with starting Rotator position -45 deg with velocity +0.02 deg/s for 30s  
Final position: ~-44.2 deg (assumes ~0.2 deg of “tracking” during the time it takes to slew from 0 to -45, note the tracking speed here is twice the one used on the 0 to 80 degrees above, which is why the buffer is similar).

In [ ]:
target = await mtcs.find_target(el=85, az=180, mag_limit=8) # Position w/ +0.02 deg/s
print(f"Found target near {el} deg elevation and {az} deg azimuth:\n"
      f"  {target}")

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=-45)
await asyncio.sleep(30)
await mtcs.stop_tracking()

Track position 0 with velocity 0 for 2 minutes.

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=0)
await asyncio.sleep(120)
await mtcs.stop_tracking()

Track target with starting Rotator position +45 deg with velocity -0.02 deg/s for 30s.
Final position: ~+44.2 deg (again, assumes ~0.2 deg of “tracking” during the time it takes to slew from 0 to +45).

In [ ]:
target = await mtcs.find_target(el=85, az=0, mag_limit=8)  # Position w/ -0.02 deg/s
print(f"Found target near {el} deg elevation and {az} deg azimuth:\n"
      f"  {target}")

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=+45)
await asyncio.sleep(30)
await mtcs.stop_tracking()

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- Starting Time: {now} UTC")

## Execute a 2-8 hours regular observing campaign.

Most of the time during operations the scheduler will be imaging a pack of the sky trying to keep the rotator close to a certain position.  
Basically we can assume it is going to “slew” the rotator to a certain position and, from there on, track and unwind the rotator constantly for hours on end.  
From the rotator perspective, this can be seen as something like:

In [ ]:
now = datetime.now()
test_message = "LVV-T2572-C - Regular Observing Campaign"
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")

---
Start with rotator position in zero.

In [ ]:
await mtcs.rem.mtrotator.cmd_move.set_start(position=0.0)

Repeat 240 times (around 2 hours):  
 - Track target with starting Rotator position around -0.3 deg and tracking velocity +0.01deg/s for 30s.

In [ ]:
target = await mtcs.find_target(el=60, az=180, mag_limit=8)  # Position w/ +0.01 deg/s
print(f"Found target near {el} deg elevation and {az} deg azimuth:\n"
      f"  {target}")

In [ ]:
for i in range(240):
    await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=-0.03)
    await asyncio.sleep(30)
    await mtcs.stop_tracking()
    # await asyncio.sleep(5) # Make sure it stops before slew again

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- Starting Time: {now} UTC")